In [1]:
import os
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import statistics

PATH_TO_DATA = '/home/gabs/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/data/WikiParser/species_texts/'
PATH_TO_META = '/home/gabs/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/data/NemuPipeline/AllInsects/midori_all_insects_mutspec/msMetaData.tsv'
PATH_TO_METAWINGS = '/home/gabs/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/data/NemuPipeline/AllInsects/midori_all_insects_mutspec/meta_wiki_wings.tsv'

In [4]:
corpus = []
for txt in os.listdir(PATH_TO_DATA):
    with open(f'{PATH_TO_DATA}{txt}') as fin:
        text = fin.read()
        corpus.append(text)


tfidf = TfidfVectorizer(lowercase=True)
res = tfidf.fit_transform(corpus)

wing_related_scores = {}
regexp = "^wing$|^wings$|^forewing$|^forewings$|^frontwing$|^frontwings$|^hindwing$|^hindwings$|^midwing$|^spreadwing$|^spreadwings$|^underwing$|^underwings$|^upperwing$|^upperwings$|^wingbeat$|^wingbeats$|^winged$|^wingspan$|^wingspans$|^wingspots$|^wingspread$|^wingtip$|^wingtips$"
for word, freq in zip(tfidf.get_feature_names_out(), tfidf.idf_):
    if re.search(regexp, word):
        wing_related_scores[word] = freq
mean = statistics.mean(wing_related_scores.values())
print(wing_related_scores) #to get keywords

{'wing': 2.337722733930307}


In [8]:
#filtering by tfidf scores, it seems like too much, though, so I'll omit it for now
wing_related_scores = {word:freq for (word, freq) in wing_related_scores.items() if freq >= mean}
regexp = '|'.join([word for word in regexp.split('|') if word.strip('[\s\.\,][\s\.\,]') in wing_related_scores.keys()]) #need to add [\s\.\,] before and after the word. manually?
regexp = '[\s\.\,]frontwings[\s\.\,]|[\s\.\,]midwing[\s\.\,]|[\s\.\,]spreadwing[\s\.\,]|[\s\.\,]spreadwings[\s\.\,]|[\s\.\,]underwings[\s\.\,]|[\s\.\,]upperwing[\s\.\,]|[\s\.\,]upperwings[\s\.\,]|[\s\.\,]wingbeat[\s\.\,]|[\s\.\,]wingbeats[\s\.\,]|[\s\.\,]wingspans[\s\.\,]|[\s\.\,]wingspots[\s\.\,]|[\s\.\,]wingspread[\s\.\,]|[\s\.\,]wingtip[\s\.\,]|[\s\.\,]wingtips[\s\.\,]'

In [22]:
#adjusting one variance of regex to match relevant words
regexp = "[a-zA-z]flies[\(\.\,\)]|[a-zA-z]fly|fly[a-zA-z]|[\(\s\.\,\)]fly[\(\s\.\,\)]|[\(\s\.\,\)]wing[\(\s\.\,\)][\(\s\.\,\)]wings[\(\s\.\,\)]|[\(\s\.\,\)]forewing[\(\s\.\,\)]|[\(\s\.\,\)]forewings[\(\s\.\,\)]|[\(\s\.\,\)]frontwing[\(\s\.\,\)]|[\(\s\.\,\)]frontwings[\(\s\.\,\)]|[\(\s\.\,\)]hindwing[\(\s\.\,\)]|[\(\s\.\,\)]hindwings[\(\s\.\,\)]|[\(\s\.\,\)]midwing[\(\s\.\,\)]|[\(\s\.\,\)]spreadwing[\(\s\.\,\)]|[\(\s\.\,\)]spreadwings[\(\s\.\,\)]|[\(\s\.\,\)]underwing[\(\s\.\,\)]|[\(\s\.\,\)]underwings[\(\s\.\,\)]|[\(\s\.\,\)]upperwing[\(\s\.\,\)]|[\(\s\.\,\)]upperwings[\(\s\.\,\)]|[\(\s\.\,\)]wingbeat[\(\s\.\,\)]|[\(\s\.\,\)]wingbeats[\(\s\.\,\)]|[\(\s\.\,\)]winged[\(\s\.\,\)]|[\(\s\.\,\)]wingspan[\(\s\.\,\)]|[\(\s\.\,\)]wingspans[\(\s\.\,\)]|[\(\s\.\,\)]wingspots[\(\s\.\,\)]|[\(\s\.\,\)]wingspread[\(\s\.\,\)]|[\(\s\.\,\)]wingtip[\(\s\.\,\)]|[\(\s\.\,\)]wingtips[\(\s\.\,\)]"

In [23]:
meta_df = pd.read_csv(PATH_TO_META, sep='\t')
meta_df['Wings'] = np.nan

#creating db
for txt in os.listdir(PATH_TO_DATA):
    with open(f'{PATH_TO_DATA}{txt}') as fin:
        text = fin.read().lower()
        if re.search(regexp, text):
            meta_df.loc[meta_df['Species'] == txt.split('.')[0], 'Wings'] = '+'
        else:
            meta_df.loc[meta_df['Species'] == txt.split('.')[0], 'Wings'] = '-'

meta_df = meta_df.fillna('NoData')

In [15]:
meta_df[meta_df['Wings'] == 'NoData']

,Species,Class,Order,Family,Genus,Nodes_in_tree,Wings
0,Gorytes_quinquefasciatus,Insecta_50557,Hymenoptera_7399,Crabronidae_253718,Gorytes_421172,10.0,NoData
3,Machimus_vescus,Insecta_50557,Diptera_7147,Asilidae_50673,Machimus_247604,41.0,NoData
4,Leucophenga_orientalis,Insecta_50557,Diptera_7147,Drosophilidae_7214,Leucophenga_7329,15.0,NoData
6,Copaxa_andescens,Insecta_50557,Lepidoptera_7088,Saturniidae_7117,Copaxa_180265,24.0,NoData
10,Aphis_cytisorum,Insecta_50557,Hemiptera_7524,Aphididae_27482,Aphis_80764,12.0,NoData
...,...,...,...,...,...,...,...
11056,Cletus_bipunctatus,Insecta_50557,Hemiptera_7524,Coreidae_186376,Cletus_237098,17.0,NoData
11058,Xestia_finatimis,Insecta_50557,Lepidoptera_7088,Noctuidae_7100,Xestia_320016,14.0,NoData
11059,Culicoides_dewulfi,Insecta_50557,Diptera_7147,Ceratopogonidae_41819,Culicoides_41820,23.0,NoData
11060,Epinotia_pulsatillana,Insecta_50557,Lepidoptera_7088,Tortricidae_7139,Epinotia_29054,16.0,NoData


In [56]:
#check them on wiki for wings
wings = meta_df[meta_df['Wings'] == '+']
for _ in range(6):
    print(wings.iloc[np.random.randint(0,3407)].Species)

Coleophora_texanella
Eupithecia_tripunctaria
Ectoedemia_subbimaculella
Dichrorampha_aeratana
Cephetola_mariae
Eudonia_sudetica


In [25]:
#check them on wiki for wings
no_wings = meta_df[meta_df['Wings'] == '-']
for _ in range(6):
    print(no_wings.iloc[np.random.randint(0,781)].Species)

Chrysopa_oculata
Anopheles_daciae
Trichocorixa_verticalis
Ceratina_smaragdula
Pseudococcus_longispinus
Notiphila_scalaris


In [54]:
wings

,Species,Class,Order,Family,Genus,Nodes_in_tree,Wings
1,Argyresthia_goedartella,Insecta_50557,Lepidoptera_7088,Argyresthiidae_572717,Argyresthia_572706,26.0,+
8,Rhopobota_naevana,Insecta_50557,Lepidoptera_7088,Tortricidae_7139,Rhopobota_572891,242.0,+
9,Perimede_falcata,Insecta_50557,Lepidoptera_7088,Cosmopterigidae_173647,Perimede_691670,10.0,+
11,Asota_eusemioides,Insecta_50557,Lepidoptera_7088,Erebidae_695564,Asota_95192,21.0,+
13,Papilio_helenus,Insecta_50557,Lepidoptera_7088,Papilionidae_7143,Papilio_7145,11.0,+
...,...,...,...,...,...,...,...
11032,Hypercompe_caudata,Insecta_50557,Lepidoptera_7088,Erebidae_695564,Hypercompe_505956,11.0,+
11044,Celaenorrhinus_proxima,Insecta_50557,Lepidoptera_7088,Hesperiidae_40093,Celaenorrhinus_354837,14.0,+
11049,Phyllonorycter_nipigon,Insecta_50557,Lepidoptera_7088,Gracillariidae_98966,Phyllonorycter_173584,31.0,+
11062,Garrha_costimacula,Insecta_50557,Lepidoptera_7088,Oecophoridae_57992,Garrha_1369220,11.0,+


TESTING


In [2]:
corpus = []
for txt in os.listdir(PATH_TO_DATA):
    with open(f'{PATH_TO_DATA}{txt}') as fin:
        text = fin.read()
        corpus.append(text)


tfidf = TfidfVectorizer(lowercase=True)
res = tfidf.fit_transform(corpus)


In [8]:
diapause_scores = {}
for word, freq in zip(tfidf.get_feature_names_out(), tfidf.idf_):
    if re.search('^diapause$', word):
        diapause_scores[word] = freq

In [9]:
meta_df = pd.read_csv(PATH_TO_META, sep='\t')
meta_df['Diapause'] = np.nan

#creating db
for txt in os.listdir(PATH_TO_DATA):
    with open(f'{PATH_TO_DATA}{txt}') as fin:
        text = fin.read().lower()
        if re.search('diapause', text):
            meta_df.loc[meta_df['Species'] == txt.split('.')[0], 'Diapause'] = '+'
        else:
            meta_df.loc[meta_df['Species'] == txt.split('.')[0], 'Diapause'] = '-'

meta_df = meta_df.fillna('NoData')

In [12]:
meta_df[meta_df['Diapause'] == '+']

,Species,Class,Order,Family,Genus,Nodes_in_tree,Diapause
133,Psychoda_alternata,Insecta_50557,Diptera_7147,Psychodidae_7197,Psychoda_7201,191.0,+
224,Lycaena_dispar,Insecta_50557,Lepidoptera_7088,Lycaenidae_27544,Lycaena_265359,17.0,+
386,Hypolimnas_bolina,Insecta_50557,Lepidoptera_7088,Nymphalidae_33415,Hypolimnas_76214,23.0,+
931,Malacosoma_californicum,Insecta_50557,Lepidoptera_7088,Lasiocampidae_7096,Malacosoma_7097,65.0,+
938,Megachile_rotundata,Insecta_50557,Hymenoptera_7399,Megachilidae_124286,Megachile_132116,46.0,+
...,...,...,...,...,...,...,...
10703,Halictus_ligatus,Insecta_50557,Hymenoptera_7399,Halictidae_77572,Halictus_77574,49.0,+
10724,Anastrepha_suspensa,Insecta_50557,Diptera_7147,Tephritidae_7211,Anastrepha_28585,18.0,+
10766,Lestes_dryas,Insecta_50557,Odonata_6961,Lestidae_126180,Lestes_181491,19.0,+
10871,Epiblema_scudderiana,Insecta_50557,Lepidoptera_7088,Tortricidae_7139,Epiblema_111440,26.0,+
